In [36]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content/yolov5/FastSAM


In [37]:
%cd {HOME}

# install FastSAM
!git clone https://github.com/CASIA-IVA-Lab/FastSAM.git
!pip -q install -r FastSAM/requirements.txt
# install CLIP
!pip -q install git+https://github.com/openai/CLIP.git
# install SAM
!pip -q install git+https://github.com/facebookresearch/segment-anything.git
# install other dependencies
!pip -q install roboflow supervision jupyter_bbox_widget

/content/yolov5/FastSAM
Cloning into 'FastSAM'...
remote: Enumerating objects: 1329, done.
remote: Counting objects: 100% (494/494), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 1329 (delta 377), reused 399 (delta 338), pack-reused 835
Receiving objects: 100% (1329/1329), 72.57 MiB | 43.74 MiB/s, done.
Resolving deltas: 100% (530/530), done.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [38]:
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://huggingface.co/spaces/An-619/FastSAM/resolve/main/weights/FastSAM.pt
!wget -P {HOME}/weights -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
!ls -lh {HOME}/weights

total 2.6G
-rw-r--r-- 1 root root 139M Jun 20  2023 FastSAM.pt
-rw-r--r-- 1 root root 2.4G Apr  4  2023 sam_vit_h_4b8939.pth


In [39]:
FAST_SAM_CHECKPOINT_PATH = f"{HOME}/weights/FastSAM.pt"
SAM_SAM_CHECKPOINT_PATH = f"{HOME}/weights/sam_vit_h_4b8939.pth"

In [2]:
#clone YOLOv5 and
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 16772, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 16772 (delta 39), reused 45 (delta 20), pack-reused 16679
Receiving objects: 100% (16772/16772), 15.39 MiB | 27.65 MiB/s, done.
Resolving deltas: 100% (11518/11518), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.2/800.2 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you hav

In [3]:
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="qmXdlsKqVuURfIBX1JvQ")
project = rf.workspace("proj-sfl83").project("grasp-6")
version = project.version(7)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to /content/datasets/grasp-6-7 in yolov5pytorch:: 100%|██████████| 6184/6184 [00:01<00:00, 5278.21it/s]


In [5]:
!python train.py --img 416 --batch 16 --epochs 30 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache

2024-07-12 17:05:52.534207: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 17:05:52.534258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 17:05:52.535908: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/datasets/grasp-6-7/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=Fal

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!mkdir /content/test_images/
!cp /content/drive/MyDrive/test_images/* /content/test_images/

In [43]:
!pip install ultralytics

In [44]:
%cd {HOME}/FastSAM
import os
import cv2
import torch
import roboflow
import base64

import supervision as sv
import numpy as np

from roboflow import Roboflow
from fastsam import FastSAM
# Load your YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/yolov5/runs/train/exp/weights/best.pt')

# Load your test images
img_paths = ['/content/test_images/IMG_1246.JPG', '/content/test_images/IMG_1246.JPG']
images = [cv2.imread(img_path) for img_path in img_paths]

# Get bounding boxes from YOLOv5
results = model(images)
bboxes = results.xyxy  # bounding boxes

# Step 3: Run FastSAM with Bounding Boxes



# Initialize FastSAM
fast_sam = fastsam()

# Function to convert YOLOv5 bounding boxes to FastSAM format
def yolo_to_fastsam_bbox(yolo_bbox):
    # Convert YOLOv5 bbox (x1, y1, x2, y2) to FastSAM bbox (x, y, width, height)
    x1, y1, x2, y2 = yolo_bbox
    return [x1, y1, x2 - x1, y2 - y1]

# Segment each image using FastSAM
segmented_images = []
for img, bbox in zip(images, bboxes):
    fast_sam_bbox = yolo_to_fastsam_bbox(bbox)
    mask = fast_sam.segment(img, bbox=fast_sam_bbox)
    segmented_images.append(mask)

# Step 4: Visualize the Segmented Images
import matplotlib.pyplot as plt

def plot_images_with_masks(images, masks):
    for i, (img, mask) in enumerate(zip(images, masks)):
        plt.figure(figsize=(10, 5))

        plt.subplot(1, 2, 1)
        plt.imshow(img)
        plt.title("Original Image")

        plt.subplot(1, 2, 2)
        plt.imshow(mask, cmap='gray')
        plt.title("Segmented Mask")

        plt.show()

plot_images_with_masks(images, segmented_images)

/content/yolov5/FastSAM/FastSAM


ModuleNotFoundError: No module named 'ultralytics.yolo'